## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np


# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Castro,CL,2019-08-26 17:25:49,-42.48,-73.76,48.20,61,40,14.99,scattered clouds,0.000,0
1,1,Lebu,ET,2019-08-26 17:25:49,8.96,38.73,58.69,83,72,1.45,light rain,2.187,0
2,2,Sitka,US,2019-08-26 17:25:49,37.17,-99.65,90.00,46,6,21.00,clear sky,0.000,0
3,3,Torbay,CA,2019-08-26 17:25:49,47.66,-52.73,75.00,43,20,10.29,few clouds,0.000,0
4,4,Grandview,US,2019-08-26 17:25:50,38.89,-94.53,69.80,93,40,4.70,heavy intensity rain,0.000,0


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 70


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
prefered_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
prefered_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
4,4,Grandview,US,2019-08-26 17:25:50,38.89,-94.53,69.80,93,40,4.70,heavy intensity rain,0.000,0
8,8,Great Falls,US,2019-08-26 17:25:51,47.50,-111.29,63.00,58,1,2.39,clear sky,0.000,0
21,21,Luderitz,NaN,2019-08-26 17:25:56,-26.65,15.16,62.60,72,0,23.04,clear sky,0.000,0
43,43,Rikitea,PF,2019-08-26 17:26:02,-23.12,-134.97,68.77,63,6,11.18,clear sky,0.000,0
66,66,Simao,CN,2019-08-26 17:26:09,22.76,100.79,66.97,97,90,0.47,light rain,1.813,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,539,Bouar,CF,2019-08-26 17:28:21,5.94,15.59,66.79,95,96,1.66,overcast clouds,0.000,0
545,545,Lima,PE,2019-08-26 17:28:21,-12.06,-77.04,62.60,77,90,8.05,overcast clouds,0.000,0
556,556,Estevan,CA,2019-08-26 17:28:26,49.14,-102.99,69.80,52,1,20.80,clear sky,0.000,0
559,559,Rovaniemi,FI,2019-08-26 17:23:49,66.50,25.72,60.80,67,0,3.36,clear sky,0.000,0


In [5]:
# 4a. Determine if there are any empty rows.
prefered_cities_df.isnull().sum()

City_ID                     0
City                        0
Country                     4
Date                        0
Lat                         0
Lng                         0
Max Temp                    0
Humidity                    0
Cloudiness                  0
Wind Speed                  0
Current Description         0
Rain inches (last 3 hrs)    0
Snow inches (last 3 hrs)    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = prefered_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Grandview,US,69.80,heavy intensity rain,38.89,-94.53,
8,Great Falls,US,63.00,clear sky,47.50,-111.29,
43,Rikitea,PF,68.77,clear sky,-23.12,-134.97,
66,Simao,CN,66.97,light rain,22.76,100.79,
70,Codrington,AU,62.60,scattered clouds,-28.95,153.24,
72,Umea,SE,68.00,clear sky,63.83,20.26,
87,Port Alfred,ZA,63.00,overcast clouds,-33.59,26.89,
104,San Juan,AR,69.49,clear sky,-31.54,-68.53,
109,Timra,SE,66.20,clear sky,62.49,17.33,
110,Mareeba,AU,62.60,scattered clouds,-16.99,145.42,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... Skipping")
        

Hotel not found ... Skipping
Hotel not found ... Skipping
Hotel not found ... Skipping
Hotel not found ... Skipping
Hotel not found ... Skipping
Hotel not found ... Skipping


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Grandview,US,69.80,heavy intensity rain,38.89,-94.53,Holiday Inn Express & Suites Kansas City-Grand...
8,Great Falls,US,63.00,clear sky,47.50,-111.29,Extended Stay America - Great Falls - Missouri...
43,Rikitea,PF,68.77,clear sky,-23.12,-134.97,People ThankYou
66,Simao,CN,66.97,light rain,22.76,100.79,Yunling Mountain Villa
70,Codrington,AU,62.60,scattered clouds,-28.95,153.24,
...,...,...,...,...,...,...,...
539,Bouar,CF,66.79,overcast clouds,5.94,15.59,Motel de l'UNITE
545,Lima,PE,62.60,overcast clouds,-12.06,-77.04,Gran Hotel Bolivar Lima
556,Estevan,CA,69.80,clear sky,49.14,-102.99,"Motel 6 Estevan, SK"
559,Rovaniemi,FI,60.80,clear sky,66.50,25.72,Arctic City Hotel


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<d1>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>


"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0,31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))